In [1]:
import pytest,os,sys,glob,shutil,shlex,subprocess,matplotlib,re,paramiko,argparse,time,pandas
from numpy.random import randint
import numpy as np
import ast

In [2]:
os.environ['PYTHONUNBUFFERED'] = '1'

ipaddrs_range="[224.0.0.1,224.0.0.3]"
gw_ip="198.168.1.1"
gw_mac="d8:a7:56:a0:71:fa"
ssid_2g="Iheb_2.4ghz"
ssid_5g="Iheb_5ghz"
wifi_password="orangepark612"
server_mac="dc:a6:32:81:9d:87"
username_server="ihebserver"
password_server="iheeb"
client_mac="dc:a6:32:ba:a9:a6"
username_client="iheb"
password_client="iheeb0"
server_ip="192.168.1.173"
client_ip="192.168.1.172"
settings_id="1"
ports=[str(randint(0,1024))]

range_splitted=ipaddrs_range.split('.')
ipaddrs=["224.0.0."+str(i) for i in range(int(range_splitted[3][:(range_splitted[3]).index(',')]),(int(range_splitted[-1][:-1])+1))]
print(ports,ipaddrs)

In [3]:
settings_id=sys.argv[1][-1]
if (settings_id=='1'):
    parameter=["settings_id","CSV_filename","ipaddrs_range","ports_range","client_ip","username_client","password_client","server_ip","username_server","password_server","ssid_2g","ssid_5g","wifi_password"]
else:    
    parameter=["settings_id","CSV_filename","ipaddrs_range","ports_range","server_mac","username_server","password_server","client_mac","username_client","password_client","ssid_2g","ssid_5g","wifi_password"]
    
parser = argparse.ArgumentParser()


for i in range(len(parameter)):
    parser.add_argument('-'+parameter[i])
namespace = parser.parse_args()
locals().update(namespace.__dict__)


def check_parameter(key,value):
    if (value== None):
        sys.exit("-"+key+"parameter has to be set!")
def check_all():
    """
    Check that all the required parameters are set
    """  
    required_parameters=namespace.__dict__
    required_parameters.pop('ports_range',"")
    for k,v in required_parameters:
        check_parameter(k,v)


if (ports_range!=None):
    ports_range=ast.literal_eval(ports_range)
    ports=list(np.arange(ports_range[0],(ports_range[1])+1))
    ports=[str(port) for port in ports ]
else:
    ports=[str(randint(0,1024))]

range_splitted=ipaddrs_range.split('.')
ipaddrs=["224.0.0."+str(i) for i in range(int(range_splitted[3][:(range_splitted[3]).index(',')]),int(range_splitted[-1][:-1])+1)]

SyntaxError: unmatched ')' (<ipython-input-3-ba1fc804c994>, line 23)

In [22]:
def rarp(mac):
    bcast_ping="ping -b 192.168.1.255 -c 1"
    subprocess.call(shlex.split(bcast_ping))   
    get_arp_table="arp -a"   
    grep_ip=" grep "+mac
    arp_table=subprocess.Popen(shlex.split(get_arp_table),stdout=subprocess.PIPE)
    output=str((subprocess.Popen(shlex.split(grep_ip),stdin=arp_table.stdout, stdout=subprocess.PIPE)).communicate())
    return output[output.find("1"):output.find(")")]
if(settings_id=="2"):
    server_ip=rarp(server_mac)
    client_ip=rarp(client_mac)    

NameError: name 'settings_id' is not defined

In [24]:
client_ip

'192.168.1.172'

In [25]:
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(client_ip, username=username_client, password=password_client)

In [26]:
ssh_server = paramiko.SSHClient()
ssh_server.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_server.connect(server_ip, username=username_server, password=password_server)

In [27]:
from socket import timeout

In [30]:
def wait4response(shell):
    while not shell.recv_ready():
        time.sleep(0.5)
    shell.recv(100000)

def wait4expected(shell,expected,error):
    shell.settimeout(90)
    try:
        recovered=str(shell.recv(100000))
        while (((recovered.find(expected))==(-1)) and ((recovered.find(error))==(-1))):
            time.sleep(0.5)
            recovered=str(shell.recv(100000))
        if(recovered.find(error)!=(-1)):
            sys.exit("Error: "+error)
        return recovered
    except timeout:
        sys.exit("Timeout error: "+error)
#     shell.settimeout(None)
    
    
def connectWifi(ssid):
        return "nmcli dev wifi connect "+ssid+" password "+wifi_password+"\n"

def no_autoconnect(ssid):
        return "nmcli con modify "+ssid+" autoconnect no\n"


combinations=["Eth-Eth","Eth-2.4GHZ","Eth-5GHZ","2.4GHZ-Eth","2.4GHZ-2.4GHZ","2.4GHZ-5GHZ","5GHZ-Eth","5GHZ-2.4GHZ","5GHZ-5GHZ"]

server_shell =ssh_server.invoke_shell()
time.sleep(1)
wait4response(server_shell)


client_shell =ssh_client.invoke_shell()
time.sleep(1)
wait4response(client_shell)

def shellnWifi_prepare(shell,passwd):
    shell.send("sudo su\n")
    wait4response(shell)

    shell.send(passwd+"\n")
    wait4response(shell)

    wifion="nmcli r wifi on\n"
    shell.send(wifion)
    wait4response(shell)

    rescan="nmcli device wifi rescan\n"
    shell.send(rescan)
    wait4response(shell)

    if(settings_id=="2"):
        shell.send(connectWifi(ssid_2g))
        wait4expected(shell,"successfully activated","2.4Ghz connection activation failed. Try to manually connect to the wifi or change the wifi channel and try again.")

        shell.send(connectWifi(ssid_5g))
        wait4expected(shell,"successfully activated","5Ghz connection activation failed. Try to manually connect to the wifi or change the wifi channel and try again.")

    shell.send(no_autoconnect(ssid_2g))
    shell.send(no_autoconnect(ssid_5g))

shellnWifi_prepare(client_shell,password_client)   
shellnWifi_prepare(server_shell,password_server)


def connect(shell,ssid=-1):
    if(ssid!=(-1)):
        shell.send("nmcli con up "+ssid+"\n")
        wait4expected(shell,"successfully activated",ssid+' activation failed')
    else:
        shell.send("sudo nmcli con down $(nmcli con show -a )\n")
        wait4expected(shell,"Error","No Error is expected")
        #wait4expected(shell,"active connections found","No connection specified.")
            

In [51]:
def wait4expected(shell,expected,error):
    shell.settimeout(1)
    try:
        recovered=str(shell.recv(100000))
        while (((recovered.find(expected))==(-1)) and ((recovered.find(error))==(-1))):
            time.sleep(0.5)
            recovered=str(shell.recv(100000))
        if(recovered.find(error)!=(-1)):
            sys.exit("Error:"+error)
        print(recovered)
    except timeout:
        sys.exit("Timeout error:"+error)

In [52]:
client_shell.send("nmcli con up "+ssid_2g+"\n")

25

In [53]:
wait4expected(client_shell,"successfully activated",ssid_2g+'activation failed')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.8/site-packages/paramiko/channel.py", line 699, in recv
    out = self.in_buffer.read(nbytes, self.timeout)
  File "/root/anaconda3/lib/python3.8/site-packages/paramiko/buffered_pipe.py", line 164, in read
    raise PipeTimeout()
paramiko.buffered_pipe.PipeTimeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-51-99cd3080de3a>", line 7, in wait4expected
    recovered=str(shell.recv(100000))
  File "/root/anaconda3/lib/python3.8/site-packages/paramiko/channel.py", line 701, in recv
    raise socket.timeout()
socket.timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-067c1c6f0d33>", line 1, i

TypeError: object of type 'NoneType' has no len()

In [48]:
client_shell.recv(100000)

b'root@iheb:/home/iheb# nmcli con up Iheb_5ghz\r\n'

In [49]:
client_shell.recv(100000)

b'Connection successfully activated (D-Bus active path: /org/freedesktop/NetworkManager/ActiveConnection/18)\r\n'

In [32]:
def test_oneCombination(combinationIndex):
    csv_abspath=os.getcwd()+"/"+CSV_filename
    pandas.DataFrame([[["",combinations[combinationIndex],""]]]).to_csv(csv_abspath,index=False,header=False,mode="a")
    pandas.DataFrame([["ip address","port","loss rate(in %)"]]).to_csv(csv_abspath,index=False,header=False,mode="a")
    for multicastip in ipaddrs:
        for port in ports:
            server="iperf -s -u -B "+multicastip+" -p "+port+" & sleep 4\n"
            server_shell.send(server)
            wait4response(server_shell)
            
            client="iperf -c "+multicastip+" -u -t3 -i1 -T999 -p "+port+"\n"
            ssh_client.exec_command(client)
            
            result=wait4expected(server_shell,'%)','159')
            print(result)
            print("\n\n")
            loss_rate=result[result.rfind("(")+1:result.rfind(")")-1]
            pandas.DataFrame([[multicastip,port,loss_rate]]).to_csv(csv_abspath,index=False,header=False,mode="a")
            
# def test_allCombinations0():
#     combinationIndex=0
#     for ssid_s in [-1]:
#         connect(server_shell,ssid_s)   
#         for ssid_c in [-1]:
#             connect(client_shell,ssid_c)
#             test_oneCombination(combinationIndex)
#             combinationIndex+=1
# def test_allCombinations1():
#     combinationIndex=0
#     for ssid_s in [-1,ssid_5g]:
#         connect(server_shell,ssid_s)   
#         for ssid_c in [-1,ssid_5g]:
#             connect(client_shell,ssid_c)
#             test_oneCombination(combinationIndex)
#             combinationIndex+=1
            
def test_allCombinations():
    combinationIndex=0
    for ssid_s in [-1,ssid_2g,ssid_5g]:
        connect(server_shell,ssid_s)   
        for ssid_c in [-1,ssid_2g,ssid_5g]:
            connect(client_shell,ssid_c)
            test_oneCombination(combinationIndex)
            combinationIndex+=1
            
test_allCombinations()

b'[ ID] Interval       Transfer     Bandwidth        Jitter   Lost/Total Datagrams\r\n[  3]  0.0- 3.0 sec   385 KBytes  1.05 Mbits/sec   0.035 ms    0/  268 (0%)\r\n'



b'[ ID] Interval       Transfer     Bandwidth        Jitter   Lost/Total Datagrams\r\n[  3]  0.0- 3.0 sec   385 KBytes  1.05 Mbits/sec   0.033 ms    0/  268 (0%)\r\n'



b'[  3]  0.0- 3.0 sec   385 KBytes  1.05 Mbits/sec   0.020 ms    0/  268 (0%)\r\n[  4]  0.0- 3.0 sec   385 KBytes  1.05 Mbits/sec   0.020 ms    0/  268 (0%)\r\n'



b'[ ID] Interval       Transfer     Bandwidth        Jitter   Lost/Total Datagrams\r\n[  3]  0.0- 3.0 sec   385 KBytes  1.05 Mbits/sec   0.032 ms    0/  268 (0%)\r\n'



b'[  3]  0.0- 3.0 sec   769 KBytes  2.12 Mbits/sec  13.438 ms    0/  268 (0%)\r\n[  3] 0.0000-2.9667 sec  268 datagrams received out-of-order'



b'iperf -s -u -B 224.0.0.2 -p 582 & sleep 4\r\n[  4]  0.0- 3.0 sec   769 KBytes  2.10 Mbits/sec  13.438 ms    0/  268 (0%)\r\n[  4] 0.0000-3.0031 sec  268 datagrams received out-o

In [138]:
ssh_server.close()
ssh_client.close()